In [136]:
import pandas as pd
import numpy as np
import datetime
import math

# this is so stupid but need to deal with automatic
# excel datetime conversion messing up the data
def datetime_to_code(date):
    if type(date) is datetime.datetime:
        return str(date.month) + '-' + str(date.year)
    else:
        return date

def format_df(df):
    # redo the indexing and manually select the rows/columns we want
    df.columns = df.iloc[2528]
    df = df.iloc[2529:3303]
    df = df[['Washington SOT', 'SOC', 'Type', 'Level']]

    # fix excel's mistakes and drop empty rows
    df['SOC'] = df['SOC'].apply(datetime_to_code)
    df = df.dropna(axis='rows')
    
    return df

def gen_mappings(df):
    # generate mappings
    mapping = {}
    for i in range(df.shape[0]):
        arr = np.zeros(7)
        level = df.iloc[i]['Level']
        if type(level) is int:
            arr[level] = 1
        else:
            arr[math.floor(level)] = 0.5
            arr[math.ceil(level)] = 0.5
        mapping[df.iloc[i]['Washington SOT']] = arr.tolist()
    return mapping

# write to yaml file
def create_levels(df, fname):
    mapping = gen_mappings(df)
    with open(fname, 'w') as yaml_file:
        yaml.dump(mapping, yaml_file, default_flow_style=None)
    return None

df = pd.read_excel('../../../covid-surge-who.xlsx', parse_dates=True)
df = format_df(df)
create_levels(df, 'levels.yaml')